# Final Project MSDS 460 : Decision Analytics

Supply Chain distribution over a 10 week horizon, modelled as a Flow model.
Source is the Warehouse supplied from Factory and destinations are the various local large retailer shops 

Each week's projections sent out to the Company warehouse by each large retailer
Products are shipped from theWarehouse to retailers

Each retailer uses aprt of the shipment to fulfill current week's orders. part to fulfill previous weeks' accumulated Backlog

Penalties : 
       Penalties exist for not meeting current week's demands
       Penalties exist for not meeting backorders 
       
       Stoichastic risk exist for losing customers
       Stoichastic risk exist for over supply and extra storage


The headquarters then allocatesthese current and planned supplies among the regions according to each region’s “impor-tance rating” and unfulﬁlled orders from previous weeks

This is a multi-period network ﬂow problem that matches demand and supply


Cust/Week  0    1    2    3    4    5    6    7    8    9 

Customer Forecast:
F_A = [15, 17, 17, 18, 19, 19, 20, 20, 20, 22]
F_B = [62, 65, 66, 58, 54, 55, 56, 53, 54, 52]
F_C = [72, 72, 77, 71, 74, 76, 78, 74, 69, 70]

Rt = [149, 154, 160, 147, 147, 150, 154, 147, 143, 144]
St = [120, 125, 130, 125, 118, 135, 132, 138, 151, 150]






Penalty Matrix :
Cust/Penalty Current-unmet  Backorder-unmet
A              10             20
B              10             23
C              15             35


Disc_Factor = 0.99
Penalty = Disc_Factor * ( Sum of unmet*current-penalty + Sum of backorder*back-penalty ) 

After LP Optimization, we will come up with an Optimized supply Flow Matrix for A, B and C.

In this LP flow model we have we have a total of 150 Decision variables and 100 Constraints 




In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pulp import LpVariable, LpProblem, LpMaximize, LpStatus, value, LpMinimize, GLPK
from pulp import *



# Decision Variables Here are the flow of supplies to each customer for each week for 10 weeks
# Other decisons variblaes are back orders, current orders not met ,which drive the penalties
# We have a total of 150 Decision variables
# Objective function is to reduce the Penalties 
# We have a total of 100 Constraints 

# Supply to customer A for week 0


# Define Custoer Forecasted and Supply Values  
# Customer Forecasts for next 10 weeks
F_A = [15, 17, 17, 18, 19, 19, 20, 20, 20, 22]
F_B = [62, 65, 66, 58, 54, 55, 56, 53, 54, 52]
F_C = [72, 72, 77, 71, 74, 76, 78, 74, 69, 70]

# Replenishment order to plants, sume of Forecasts F(A,B,C)
R = [149, 154, 160, 147, 147, 150, 154, 147, 143, 144]


# Supply flow for the next 10 weeks from the Warehouse
S = [120, 125, 130, 125, 118, 135, 132, 138, 151, 150]

# Penalty , current (c) index 0 and BackOrder (b) index 1
Discount_Factor = 0.99
# Cust-A  
P_A = [10, 20]

# Cust-B
P_B = [10, 23]

# Cust-C
P_C = [15, 35]


# define variables
# sct, xct, yct, act, bct 

#RANGE = range(10)
#x = pulp.LpVariable.dicts("x", RANGE, LB_ind, UB_ind, "Integer")

#prob += pulp.lpSum([Prices[i]*Dispatch[i] for i in RANGE])

RANGE = range(10)

# Flow supply for Customer A over time horizon 1-10 weeks
A_s = pulp.LpVariable.dicts("A_s", RANGE, 0, None, cat='Integer')

# Amount of supply allocated to meet current week's Forecast
A_x = pulp.LpVariable.dicts("A_x", RANGE, 0, None, cat='Integer')

# Amount of supply allocated to meet sum of unmet order and back order from previous weeks 
A_y = pulp.LpVariable.dicts("A_y", RANGE, 0, None, cat='Integer')

# Amount of Forecast number, not met for current week, some penalty occurs
A_a = pulp.LpVariable.dicts("A_a", RANGE, 0, None, cat='Integer')

# Amount of Backorder not met , much higher penalty
A_b = pulp.LpVariable.dicts("A_b", RANGE, 0, None, cat='Integer')


# Flow supply for Customer B over time horizon 1-10 weeks
B_s = pulp.LpVariable.dicts("B_s", RANGE, 0, None, cat='Integer')

# Amount of supply allocated to meet current week's Forecast
B_x = pulp.LpVariable.dicts("B_x", RANGE, 0, None, cat='Integer')

# Amount of supply allocated to meet sum of unmet order and back order from previous weeks 
B_y = pulp.LpVariable.dicts("B_y", RANGE, 0, None, cat='Integer')

# Amount of Forecast number, not met for current week, some penalty occurs
B_a = pulp.LpVariable.dicts("B_a", RANGE, 0, None, cat='Integer')

# Amount of Backorder not met , much higher penalty
B_b = pulp.LpVariable.dicts("B_b", RANGE, 0, None, cat='Integer')


# Flow supply for Customer C over time horizon 1-10 weeks
C_s = pulp.LpVariable.dicts("C_s", RANGE, 0, None, cat='Integer')

# Amount of supply allocated to meet current week's Forecast
C_x = pulp.LpVariable.dicts("C_x", RANGE, 0, None, cat='Integer')

# Amount of supply allocated to meet sum of unmet order and back order from previous weeks 
C_y = pulp.LpVariable.dicts("C_y", RANGE, 0, None, cat='Integer')

# Amount of Forecast number, not met for current week, some penalty occurs
C_a = pulp.LpVariable.dicts("C_a", RANGE, 0, None, cat='Integer')

# Amount of Backorder not met , much higher penalty
C_b = pulp.LpVariable.dicts("C_b", RANGE, 0, None, cat='Integer')



# defines the problem
probSupplyFlow = LpProblem("problem", LpMinimize)
# Note, LpMaximize for a maximization problem, 
# and LpMinimize for a minimization problem

# define constraints
# prob += lpSum([x_cont[i]*costs[i] for i in range(n_vars)]) <= budget

probSupplyFlow += A_x[0] + A_a[0]  == F_A[0]
probSupplyFlow += A_x[1] + A_a[1]  == F_A[1]
probSupplyFlow += A_x[2] + A_a[2]  == F_A[2]
probSupplyFlow += A_x[3] + A_a[3]  == F_A[3]
probSupplyFlow += A_x[4] + A_a[4]  == F_A[4]
probSupplyFlow += A_x[5] + A_a[5]  == F_A[5]
probSupplyFlow += A_x[6] + A_a[6]  == F_A[6]
probSupplyFlow += A_x[7] + A_a[7]  == F_A[7]
probSupplyFlow += A_x[8] + A_a[8]  == F_A[8]
probSupplyFlow += A_x[9] + A_a[9]  == F_A[9]


probSupplyFlow += A_x[0] + A_y[0]  <= A_s[0]
probSupplyFlow += A_x[1] + A_y[1]  <= A_s[1]
probSupplyFlow += A_x[2] + A_y[2]  <= A_s[2]
probSupplyFlow += A_x[3] + A_y[3]  <= A_s[3]
probSupplyFlow += A_x[4] + A_y[4]  <= A_s[4]
probSupplyFlow += A_x[5] + A_y[5]  <= A_s[5]
probSupplyFlow += A_x[6] + A_y[6]  <= A_s[6]
probSupplyFlow += A_x[7] + A_y[7]  <= A_s[7]
probSupplyFlow += A_x[8] + A_y[8]  <= A_s[8]
probSupplyFlow += A_x[9] + A_y[9]  <= A_s[9]

# yt + bt -( at-1 + bt-1 ) == 0 
probSupplyFlow += A_y[1] + A_b[1] - ( A_a[0] + A_b[0] ) == 0
probSupplyFlow += A_y[2] + A_b[2] - ( A_a[1] + A_b[1] ) == 0 
probSupplyFlow += A_y[3] + A_b[3] - ( A_a[2] + A_b[2] ) == 0 
probSupplyFlow += A_y[4] + A_b[4] - ( A_a[3] + A_b[3] ) == 0 
probSupplyFlow += A_y[5] + A_b[5] - ( A_a[4] + A_b[4] ) == 0 
probSupplyFlow += A_y[6] + A_b[6] - ( A_a[5] + A_b[5] ) == 0 
probSupplyFlow += A_y[7] + A_b[7] - ( A_a[6] + A_b[6] ) == 0 
probSupplyFlow += A_y[8] + A_b[8] - ( A_a[7] + A_b[7] ) == 0 
probSupplyFlow += A_y[9] + A_b[9] - ( A_a[8] + A_b[8] ) == 0 


probSupplyFlow += B_x[0] + B_a[0]  == F_B[0]
probSupplyFlow += B_x[1] + B_a[1]  == F_B[1]
probSupplyFlow += B_x[2] + B_a[2]  == F_B[2]
probSupplyFlow += B_x[3] + B_a[3]  == F_B[3]
probSupplyFlow += B_x[4] + B_a[4]  == F_B[4]
probSupplyFlow += B_x[5] + B_a[5]  == F_B[5]
probSupplyFlow += B_x[6] + B_a[6]  == F_B[6]
probSupplyFlow += B_x[7] + B_a[7]  == F_B[7]
probSupplyFlow += B_x[8] + B_a[8]  == F_B[8]
probSupplyFlow += B_x[9] + B_a[9]  == F_B[9]


probSupplyFlow += B_x[0] + B_y[0]  <= B_s[0]
probSupplyFlow += B_x[1] + B_y[1]  <= B_s[1]
probSupplyFlow += B_x[2] + B_y[2]  <= B_s[2]
probSupplyFlow += B_x[3] + B_y[3]  <= B_s[3]
probSupplyFlow += B_x[4] + B_y[4]  <= B_s[4]
probSupplyFlow += B_x[5] + B_y[5]  <= B_s[5]
probSupplyFlow += B_x[6] + B_y[6]  <= B_s[6]
probSupplyFlow += B_x[7] + B_y[7]  <= B_s[7]
probSupplyFlow += B_x[8] + B_y[8]  <= B_s[8]
probSupplyFlow += B_x[9] + B_y[9]  <= B_s[9]



probSupplyFlow += B_y[1] + B_b[1] - ( B_a[0] + B_b[0] ) == 0
probSupplyFlow += B_y[2] + B_b[2] - ( B_a[1] + B_b[1] ) == 0
probSupplyFlow += B_y[3] + B_b[3] - ( B_a[2] + B_b[2] ) == 0
probSupplyFlow += B_y[4] + B_b[4] - ( B_a[3] + B_b[3] ) == 0
probSupplyFlow += B_y[5] + B_b[5] - ( B_a[4] + B_b[4] ) == 0
probSupplyFlow += B_y[6] + B_b[6] - ( B_a[5] + B_b[5] ) == 0
probSupplyFlow += B_y[7] + B_b[7] - ( B_a[6] + B_b[6] ) == 0
probSupplyFlow += B_y[8] + B_b[8] - ( B_a[7] + B_b[7] ) == 0
probSupplyFlow += B_y[9] + B_b[9] - ( B_a[8] + B_b[8] ) == 0

# Customer C
probSupplyFlow += C_x[0] + C_a[0]  == F_C[0]
probSupplyFlow += C_x[1] + C_a[1]  == F_C[1]
probSupplyFlow += C_x[2] + C_a[2]  == F_C[2]
probSupplyFlow += C_x[3] + C_a[3]  == F_C[3]
probSupplyFlow += C_x[4] + C_a[4]  == F_C[4]
probSupplyFlow += C_x[5] + C_a[5]  == F_C[5]
probSupplyFlow += C_x[6] + C_a[6]  == F_C[6]
probSupplyFlow += C_x[7] + C_a[7]  == F_C[7]
probSupplyFlow += C_x[8] + C_a[8]  == F_C[8]
probSupplyFlow += C_x[9] + C_a[9]  == F_C[9]



probSupplyFlow += C_x[0] + C_y[0]  == C_s[0]
probSupplyFlow += C_x[1] + C_y[1]  == C_s[1]
probSupplyFlow += C_x[2] + C_y[2]  == C_s[2]
probSupplyFlow += C_x[3] + C_y[3]  == C_s[3]
probSupplyFlow += C_x[4] + C_y[4]  == C_s[4]
probSupplyFlow += C_x[5] + C_y[5]  == C_s[5]
probSupplyFlow += C_x[6] + C_y[6]  == C_s[6]
probSupplyFlow += C_x[7] + C_y[7]  == C_s[7]
probSupplyFlow += C_x[8] + C_y[8]  == C_s[8]
probSupplyFlow += C_x[9] + C_y[9]  == C_s[9]



probSupplyFlow += C_y[1] + C_b[1] - ( C_a[0] + C_b[0] ) == 0
probSupplyFlow += C_y[2] + C_b[2] - ( C_a[1] + C_b[1] ) == 0
probSupplyFlow += C_y[3] + C_b[3] - ( C_a[2] + C_b[2] ) == 0
probSupplyFlow += C_y[4] + C_b[4] - ( C_a[3] + C_b[3] ) == 0
probSupplyFlow += C_y[5] + C_b[5] - ( C_a[4] + C_b[4] ) == 0
probSupplyFlow += C_y[6] + C_b[6] - ( C_a[5] + C_b[5] ) == 0
probSupplyFlow += C_y[7] + C_b[7] - ( C_a[6] + C_b[6] ) == 0
probSupplyFlow += C_y[8] + C_b[8] - ( C_a[7] + C_b[7] ) == 0
probSupplyFlow += C_y[9] + C_b[9] - ( C_a[8] + C_b[8] ) == 0



probSupplyFlow += A_s[0] + B_s[0] + C_s[0] <= S[0]
probSupplyFlow += A_s[1] + B_s[1] + C_s[1] <= S[1]
probSupplyFlow += A_s[2] + B_s[2] + C_s[2] <= S[2]
probSupplyFlow += A_s[3] + B_s[3] + C_s[3] <= S[3]
probSupplyFlow += A_s[4] + B_s[4] + C_s[4] <= S[4]
probSupplyFlow += A_s[5] + B_s[5] + C_s[5] <= S[5]
probSupplyFlow += A_s[6] + B_s[6] + C_s[6] <= S[6]
probSupplyFlow += A_s[7] + B_s[7] + C_s[7] <= S[7]
probSupplyFlow += A_s[8] + B_s[8] + C_s[8] <= S[8]
probSupplyFlow += A_s[9] + B_s[9] + C_s[9] <= S[9]




#prob += lpSum([A_s[i]*costs[i] for i in RANGE]) <= budget

#probSupplyFlow += B1+B2+B3+B4 <= 3100
#probSupplyFlow += A_1+B1 <= 1800
#probSupplyFlow += A2+B2 <= 2300
#probSupplyFlow += A3+B3 <= 1250
#probSupplyFlow += A4+B4 <= 1750

# Note, if <= then <=
# If >= then >=
# If = then ==

# define objective function
#Disc_Factor = 0.99
#Penalty = Disc_Factor * ( Sum of unmet*current-penalty + Sum of backorder*back-penalty ) 

# prob += pulp.lpSum([Prices[i]*Dispatch[i] for i in RANGE])


probSupplyFlow += Discount_Factor * ( pulp.lpSum( [(A_a[i]*P_A[0] + A_b[i]*P_A[1]) for i in RANGE] ) \
                    + pulp.lpSum( [(B_a[i]*P_B[0] + B_b[i]*P_B[1]) for i in RANGE] ) \
                    + pulp.lpSum( [(C_a[i]*P_C[0] + C_b[i]*P_C[1]) for i in RANGE] ) ) 

# solve the problem
probSupplyFlow.writeLP("probSupplyFlow.lp")

#prob4.solve(GLPK(options=['--ranges prob4.sen']))
probSupplyFlow.solve(GLPK(msg=True, options=['--ranges', 'probSupplyFlow.sensitivity.txt']))

print ("Status:", LpStatus[probSupplyFlow.status])

# Note, we are only able to get sensitivity information because we are solving
# as a linear program.  If we solved as an Integer Program, then no 
# sensitivity information would be available.

for v in probSupplyFlow.variables():
    print(v.name, "=", v.varValue)

print ("Objective", round( value(probSupplyFlow.objective), 2) )
print ("")



#x_soln = np.array([x_cont[i].varValue for i in range(n_vars)])

#print("costs: ", costs)
#print("returns: ", returns)

#print("types: ", types)
#print("x_soln: ", x_soln)

                                        
                                        
                                        


GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --cpxlp /var/folders/vy/ttqf684x6j59_vcfzj1wtd7h0000gn/T/a7d3fc85337848fb8902ab990077a58d-pulp.lp
 -o /var/folders/vy/ttqf684x6j59_vcfzj1wtd7h0000gn/T/a7d3fc85337848fb8902ab990077a58d-pulp.sol
 --ranges probSupplyFlow.sensitivity.txt
Reading problem data from '/var/folders/vy/ttqf684x6j59_vcfzj1wtd7h0000gn/T/a7d3fc85337848fb8902ab990077a58d-pulp.lp'...
97 rows, 150 columns, 288 non-zeros
150 integer variables, none of which are binary
414 lines were read
GLPK Integer Optimizer, v4.65
97 rows, 150 columns, 288 non-zeros
150 integer variables, none of which are binary
Preprocessing...
97 rows, 150 columns, 288 non-zeros
150 integer variables, none of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 97
Solving LP relaxation...
GLPK Simplex Optimizer, v4.65
97 rows, 150 col